# Segundo Tech Challenge: Algoritmo Genético para Alocação de Enfermeiras em Turnos
**Aluno:** Vinícius Oliveira Litran Andrade.

## Definição do Problema: Alocação de Enfermeiras em Turnos

O problema central é a otimização da alocação de enfermeiras em turnos de trabalho ao longo de um período de tempo, considerando várias restrições e preferências. O objetivo é criar um cronograma que minimize conflitos e maximize a satisfação das enfermeiras, respeitando as limitações de carga de trabalho e as preferências individuais.

### Restrições e Preferências

* **Número de Enfermeiras e Turnos:** O código lida com um número fixo de enfermeiras e turnos por dia.
* **Limite de Turnos Semanais:** Há um limite máximo de turnos que cada enfermeira pode trabalhar por semana.
* **Preferências de Turno:** Cada enfermeira tem preferências por determinados turnos.
* **Disponibilidade:** As enfermeiras podem ter dias de indisponibilidade.
* **Dias e Turnos Consecutivos:** Há limites para o número de dias e turnos consecutivos que uma enfermeira pode trabalhar.
* **Equilíbrio na Carga de Trabalho:** A carga de trabalho deve ser distribuída de forma equilibrada entre as enfermeiras.

### Objetivos

1.  **Maximizar o Custo da Solução:** O algoritmo busca maximizar uma função de aptidão que penaliza a violação de restrições e recompensa o atendimento de preferências.
2.  **Respeitar as Restrições:** O cronograma gerado deve respeitar todas as restrições de carga de trabalho e disponibilidade.
3.  **Atender às Preferências:** O cronograma deve tentar atender às preferências de turno das enfermeiras.
4.  **Encontrar uma Solução Ótima ou Próxima do Ótimo:** O algoritmo genético busca encontrar a melhor solução possível dentro de um tempo razoável.

### Critérios de Sucesso

1.  **Aptidão da Melhor Solução:** A aptidão da melhor solução encontrada pelo algoritmo genético é um critério chave. Quanto maior a aptidão, melhor a solução.
2.  **Respeito às Restrições:** O cronograma gerado deve violar o mínimo possível de restrições.
3.  **Atendimento às Preferências:** O cronograma deve atender o máximo possível das preferências das enfermeiras.
4.  **Tempo de Execução:** O tempo que o algoritmo leva para encontrar uma solução é um critério importante. Idealmente, o algoritmo deve encontrar uma boa solução em um tempo razoável.
5.  **Equilíbrio na Carga de Trabalho:** A carga de trabalho deve ser distribuída de forma equilibrada entre as enfermeiras.
6.  **Convergência do Algoritmo:** O algoritmo deve convergir para uma solução estável após um número razoável de gerações.
7.  **Comparação com Força Bruta:** A comparação com um algoritmo de força bruta (se viável) pode ajudar a avaliar a qualidade da solução encontrada pelo algoritmo genético.


## Importação das bibliotecas

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import itertools
import time

### 1. Definição dos Parâmetros do Problema
### Configurações do problema de alocação de enfermeiras em turnos.

In [ ]:
N_ENFERMEIRAS = 4  # Número total de enfermeiras disponíveis.
N_TURNOS = 3  # Número de turnos por dia (ex: manhã, tarde, noite).
MAX_TURNOS_SEMANAIS = 5  # Limite máximo de turnos que uma enfermeira pode trabalhar por semana.
N_DIAS = 31  # Número total de dias no período de alocação.
N_CONVERGENCIA = min(20, N_DIAS)  # Número de gerações sem melhoria para considerar convergência.
POPULATION_SIZE = min(1000, 100 * N_DIAS + 10 * N_ENFERMEIRAS + 50 * N_TURNOS)  # Tamanho da população do algoritmo genético. 
TOURNAMENT_SIZE = max(2, int(0.02 * POPULATION_SIZE))  # Tamanho do torneio para seleção de pais.

### 2. Geração das Requisições de Turno
# 
### As enfermeiras têm preferências aleatórias e um limite máximo de turnos que podem trabalhar.

In [ ]:
def gerar_requisicoes_turnos(n_enfermeiras, n_dias, n_turnos):
    """Gera requisições de turnos realistas para as enfermeiras."""

    requisicoes = []  # Inicializa a lista que armazenará as requisições de turnos para cada enfermeira
    for enfermeira in range(n_enfermeiras):  # Loop para cada enfermeira
        preferencia_turno = random.randint(0, n_turnos - 1)  # Define um turno preferido aleatório para a enfermeira
        dias_consecutivos_max = random.randint(3, 5)  # Define um número máximo aleatório de dias consecutivos que a enfermeira gostaria de trabalhar, nesse caso entre 3 a 5
        turnos_consecutivos_max = 2  # Define um número máximo de turnos consecutivos que a enfermeira gostaria de trabalhar
        disponibilidade = [random.random() > 0.1 for _ in range(n_dias)]  # Gera uma lista booleana indicando a disponibilidade da enfermeira em cada dia (10% de chance de indisponibilidade)
        intensidade_requisicoes = random.uniform(0.5, 1.0)  # Define a intensidade das requisições da enfermeira (probabilidade de requisitar um turno)

        enfermeira_requisicoes = []  # Inicializa a lista que armazenará as requisições de turnos para a enfermeira atual
        dias_trabalhados = 0  # Inicializa o contador de dias consecutivos trabalhados
        turnos_trabalhados = 0  # Inicializa o contador de turnos consecutivos trabalhados

        for dia in range(n_dias):  # Loop para cada dia
            if disponibilidade[dia] and dias_trabalhados < dias_consecutivos_max:  # Verifica se a enfermeira está disponível e se não excedeu o limite de dias consecutivos
                turnos = [0] * n_turnos  # Inicializa a lista de turnos para o dia com zeros (nenhum turno requisitado)
                if random.random() < intensidade_requisicoes and turnos_trabalhados < turnos_consecutivos_max:  # Verifica se a enfermeira requisita um turno e se não excedeu o limite de turnos consecutivos
                    # Aplica preferência de turno com alguma aleatoriedade
                    turno_requerido = random.choice([preferencia_turno, random.randint(0, n_turnos - 1)])  # Escolhe o turno requisitado (preferido ou aleatório)
                    turnos[turno_requerido] = 1  # Marca o turno requisitado com 1
                    dias_trabalhados += 1  # Incrementa o contador de dias consecutivos trabalhados
                    turnos_trabalhados += 1  # Incrementa o contador de turnos consecutivos trabalhados
                else:
                    turnos_trabalhados = 0  # Reseta o contador de turnos consecutivos se não houver requisição
                enfermeira_requisicoes.append(turnos)  # Adiciona a lista de turnos do dia às requisições da enfermeira
            else:
                enfermeira_requisicoes.append([0] * n_turnos)  # Adiciona uma lista de zeros (nenhum turno requisitado) se a enfermeira estiver indisponível ou exceder o limite de dias consecutivos
                dias_trabalhados = 0  # Reseta o contador de dias consecutivos se a enfermeira estiver indisponível
                turnos_trabalhados = 0 # reseta o contador de turnos consecutivos

        requisicoes.append(enfermeira_requisicoes)  # Adiciona as requisições da enfermeira à lista de requisições
    return requisicoes

In [ ]:
requisicoes_turnos = gerar_requisicoes_turnos(N_ENFERMEIRAS, N_DIAS, N_TURNOS)
print(requisicoes_turnos)

### 3. Estratégias de Inicialização
# 
- Solução Inicial Aleatória: Gera soluções aleatórias.
- Solução Hotstart Baseada em Requisições: Gera soluções que atendem às requisições das enfermeiras.
- Solução Hotstart Balanceada: Gera soluções que balanceiam a carga de trabalho.
- Solução Hotstart com Ordem Sequencial: Gera soluções com ordem sequencial.
- População Inicial Diversificada: Combina as estratégias anteriores para criar uma população diversificada.

In [ ]:
def criar_solucao_inicial(n_enfermeiras, n_dias, n_turnos):
    """Gera uma solução inicial aleatória."""
    
    return np.random.randint(0, n_enfermeiras, size=(n_dias, n_turnos)).tolist()

#### Representação do Cromossomo:

- Cada solução (cromossomo) representa um cronograma de alocação de enfermeiras.
- A solução é representada como uma matriz bidimensional, onde as linhas representam os dias, as colunas representam os turnos e os elementos da matriz são as enfermeiras.


In [ ]:
criar_solucao_inicial(N_ENFERMEIRAS, N_DIAS, N_TURNOS)

In [ ]:
def criar_solucao_hotstart(requisicoes, n_enfermeiras, n_dias, n_turnos):
    """Gera solução inicial baseada nas requisições."""
    
    solucao = np.full((n_dias, n_turnos), -1)  # Inicializa uma matriz com -1, representando turnos não atribuídos
    for dia in range(n_dias):  # Loop para cada dia
        for turno in range(n_turnos):  # Loop para cada turno do dia
            enfermeiras_possiveis = [e for e in range(n_enfermeiras) if requisicoes[e][dia][turno] == 1]  # Lista de enfermeiras que requisitaram o turno
            solucao[dia, turno] = np.random.choice(enfermeiras_possiveis) if enfermeiras_possiveis else np.random.randint(0, n_enfermeiras)  # Atribui uma enfermeira ao turno
    return solucao.tolist()  # Retorna a solução como uma lista de listas

In [ ]:
criar_solucao_hotstart(requisicoes_turnos, N_ENFERMEIRAS, N_DIAS, N_TURNOS)

In [ ]:
def criar_solucao_hotstart_balanceada(n_enfermeiras, n_dias, n_turnos):
    """Gera uma solução inicial priorizando o balanceamento de turnos entre as enfermeiras usando NumPy."""
    
    solucao = np.zeros((n_dias, n_turnos), dtype=int)  # Inicializa uma matriz de solução com zeros
    turnos_enfermeiras = np.zeros(n_enfermeiras, dtype=int)  # Inicializa um array para contar os turnos atribuídos a cada enfermeira
    for dia in range(n_dias):  # Loop para cada dia
        for turno in range(n_turnos):  # Loop para cada turno do dia
            enfermeiras_possiveis = np.argsort(turnos_enfermeiras)  # Ordena as enfermeiras pelo número de turnos atribuídos
            enfermeira_escolhida = np.random.choice(enfermeiras_possiveis[:2])  # Escolhe aleatoriamente uma das 2 enfermeiras com menos turnos
            solucao[dia, turno] = enfermeira_escolhida  # Atribui a enfermeira escolhida ao turno
            turnos_enfermeiras[enfermeira_escolhida] += 1  # Incrementa o contador de turnos da enfermeira escolhida
    return solucao.tolist()  # Retorna a solução como uma lista de listas

In [ ]:
criar_solucao_hotstart_balanceada(N_ENFERMEIRAS, N_DIAS, N_TURNOS)

In [ ]:
def hotstart_ordem_enfermeiras_com_dias(n_enfermeiras, n_dias, n_turnos):
    """Gera uma solução inicial (hotstart) em que as enfermeiras são alocadas de forma sequencial nos turnos de cada dia, considerando a quantidade de dias e turnos. """

    # Inicializa a alocação com -1 (sem turno atribuído)
    alocacao = np.full((n_dias, n_turnos), -1)

    # A cada dia, vamos atribuindo os turnos para as enfermeiras, seguindo uma ordem sequencial
    enfermeira_atual = 0

    for dia in range(n_dias):
        for turno in range(n_turnos):
            alocacao[dia, turno] = enfermeira_atual
            enfermeira_atual = (enfermeira_atual + 1) % n_enfermeiras  # Passa para a próxima enfermeira (circular)

    return alocacao.tolist()

In [ ]:
hotstart_ordem_enfermeiras_com_dias(N_ENFERMEIRAS, N_DIAS, N_TURNOS)

In [ ]:
def criar_populacao_inicial(tamanho_populacao, n_enfermeiras, n_dias, n_turnos, requisicoes_turnos):
    """Cria a população inicial com diversidade, incluindo soluções hotstart."""

    num_hotstart = int(tamanho_populacao * 0.1) # Define o tamanho das soluções hotstart a serem geradas
    num_hotstart_ordem = 1  # Define o número de soluções hotstart com ordem sequencial a serem geradas
    populacao = []  # Inicializa a lista que armazenará a população inicial

    # Adiciona soluções hotstart baseadas em requisições
    for _ in range(num_hotstart):
        populacao.append(criar_solucao_hotstart(requisicoes_turnos, n_enfermeiras, n_dias, n_turnos))

    # Adiciona soluções hotstart balanceadas
    for _ in range(num_hotstart):
        populacao.append(criar_solucao_hotstart_balanceada(n_enfermeiras, n_dias, n_turnos))

    # Adiciona soluções hotstart com ordem sequencial
    for _ in range(num_hotstart_ordem):
        populacao.append(hotstart_ordem_enfermeiras_com_dias(n_enfermeiras, n_dias, n_turnos))

    # Completa o restante da população com soluções aleatórias
    for _ in range(tamanho_populacao - 2 * num_hotstart - num_hotstart_ordem):
        populacao.append(criar_solucao_inicial(n_enfermeiras, n_dias, n_turnos))

    return populacao

In [ ]:
população_ex = criar_populacao_inicial(POPULATION_SIZE, N_ENFERMEIRAS, N_DIAS, N_TURNOS, requisicoes_turnos)
print(população_ex)

### 4. Função de Aptidão
# 
### A função de aptidão avalia a qualidade de cada solução, considerando:
- Penalidades por violação de restrições (excesso de turnos, turnos consecutivos).
- Bonificações por atendimento de preferências de turno.
- Penalização por desequilíbrio na carga de trabalho.

In [ ]:
def calcular_aptidao(solucao, requisicoes_turnos, n_enfermeiras, n_dias, n_turnos, max_turnos_semanais):
    """Calcula a aptidão de uma solução com otimizações eficientes usando NumPy."""

    solucao = np.array(solucao)  # Converte a solução para um array NumPy para operações eficientes
    carga_trabalho = np.zeros(n_enfermeiras, dtype=int)  # Inicializa um array para contar a carga de trabalho de cada enfermeira
    turnos_semanais = np.zeros((n_enfermeiras, n_dias // 7 + 1), dtype=int)  # Inicializa um array para contar os turnos semanais de cada enfermeira
    turnos_por_enfermeira_por_dia = np.zeros((n_dias, n_enfermeiras), dtype=int)  # Inicializa um array para contar os turnos de cada enfermeira por dia

    # Pré-cálculo dos turnos semanais
    semana = np.arange(n_dias) // 7  # Calcula o número da semana para cada dia
    for dia in range(n_dias):
        for turno in range(n_turnos):
            enfermeira = solucao[dia, turno]
            if enfermeira >= 0:
                turnos_semanais[enfermeira, semana[dia]] += 1  # Incrementa o contador de turnos semanais da enfermeira

    # Pré-cálculo de turnos diários por enfermeira
    for dia in range(n_dias):
        enfermeiras_no_dia, contagem = np.unique(solucao[dia, solucao[dia] >= 0], return_counts=True)  # Conta os turnos de cada enfermeira no dia
        turnos_por_enfermeira_por_dia[dia, enfermeiras_no_dia] = contagem  # Armazena a contagem dos turnos

    # Cálculo da carga de trabalho
    carga_trabalho = np.bincount(solucao[solucao >= 0].flatten(), minlength=n_enfermeiras)  # Conta o número total de turnos de cada enfermeira

    # Penalização por excesso de turnos semanais
    excesso_semanal = np.maximum(turnos_semanais - max_turnos_semanais, 0)  # Calcula o excesso de turnos semanais
    penalidade_excesso = -3 * np.sum(excesso_semanal)  # Calcula a penalidade por excesso de turnos semanais

    # Penalização incremental para turnos excedentes por enfermeira
    turnos_excedentes = np.maximum(turnos_por_enfermeira_por_dia - 1, 0)  # Calcula o excesso de turnos por dia
    penalidade_turnos = -np.sum(turnos_excedentes * 30)  # Calcula a penalidade por excesso de turnos por dia

    # Bonificações por atendimento de requisições (otimizado com NumPy)
    solucao_np = np.array(solucao)
    requisicoes_np = np.array(requisicoes_turnos)

    bonificacao_requisicoes = np.sum((solucao_np >= 0) & (requisicoes_np[solucao_np, np.arange(n_dias)[:, None], np.arange(n_turnos)] == 1))  # Calcula a bonificação por atendimento de requisições

    # Penalização por turnos consecutivos (último turno do dia seguido do primeiro do próximo)
    penalidade_consecutivos = np.sum((solucao[:-1, -1] >= 0) & (solucao[:-1, -1] == solucao[1:, 0]))  # Calcula a penalidade por turnos consecutivos
    penalidade_turnos_dias_consecutivos_N_M = -50 * penalidade_consecutivos  # Calcula a penalidade por turnos consecutivos

    # Penalização por desequilíbrio na carga de trabalho (usando variância)
    penalidade_variancia = -10 * np.var(carga_trabalho)  # Calcula a penalidade por desequilíbrio na carga de trabalho

    return penalidade_excesso + penalidade_turnos + bonificacao_requisicoes + penalidade_turnos_dias_consecutivos_N_M + penalidade_variancia  # Retorna a aptidão total

In [ ]:
def calcular_aptidao_detalhada(solucao, requisicoes_turnos, n_enfermeiras, n_dias, n_turnos, max_turnos_semanais):
    """Calcula a aptidão de uma solução e imprime detalhadamente cada fator de pontuação."""
    
    solucao = np.array(solucao)
    carga_trabalho = np.zeros(n_enfermeiras, dtype=int)
    turnos_semanais = np.zeros((n_enfermeiras, n_dias // 7 + 1), dtype=int)
    turnos_por_enfermeira_por_dia = np.zeros((n_dias, n_enfermeiras), dtype=int)

    # Cálculo dos turnos semanais
    semana = np.arange(n_dias) // 7
    for dia in range(n_dias):
        for turno in range(n_turnos):
            enfermeira = solucao[dia, turno]
            if enfermeira >= 0:
                turnos_semanais[enfermeira, semana[dia]] += 1

    # Contagem de turnos diários por enfermeira
    for dia in range(n_dias):
        enfermeiras_no_dia, contagem = np.unique(solucao[dia, solucao[dia] >= 0], return_counts=True)
        turnos_por_enfermeira_por_dia[dia, enfermeiras_no_dia] = contagem

    # Cálculo da carga de trabalho
    carga_trabalho = np.bincount(solucao[solucao >= 0].flatten(), minlength=n_enfermeiras)
    
    # Penalização por excesso de turnos semanais
    excesso_semanal = np.maximum(turnos_semanais - max_turnos_semanais, 0)
    penalidade_excesso_semanal = -3 * np.sum(excesso_semanal)
    
    # Penalização incremental para turnos excedentes por enfermeira
    turnos_excedentes = np.maximum(turnos_por_enfermeira_por_dia - 1, 0)
    penalidade_turnos_excedentes = -np.sum(turnos_excedentes * 30)
    
    # Bonificações por atendimento de requisições (otimizado com NumPy)
    solucao_np = np.array(solucao)
    requisicoes_np = np.array(requisicoes_turnos)
    
    bonificacao_requisicoes = np.sum((solucao_np >= 0) & (requisicoes_np[solucao_np, np.arange(n_dias)[:, None], np.arange(n_turnos)] == 1))
    
    # Penalização por turnos consecutivos
    penalidade_consecutivos = -50 * np.sum((solucao[:-1, -1] >= 0) & (solucao[:-1, -1] == solucao[1:, 0]))
    
    # Penalização por desequilíbrio na carga de trabalho
    penalidade_desequilibrio = -10 * np.var(carga_trabalho)
    
    # Cálculo da aptidão total
    aptidao = (penalidade_excesso_semanal + penalidade_turnos_excedentes + bonificacao_requisicoes +
               penalidade_consecutivos + penalidade_desequilibrio)
    
    # Impressão detalhada
    print("\n=== Detalhamento da Aptidão ===")
    print(f"Penalização por excesso de turnos semanais: {penalidade_excesso_semanal}")
    print(f"Penalização por turnos excedentes no mesmo dia: {penalidade_turnos_excedentes}")
    print(f"Bonificação por atendimento de requisições: {bonificacao_requisicoes}")
    print(f"Penalização por turnos consecutivos: {penalidade_consecutivos}")
    print(f"Penalização por desequilíbrio na carga de trabalho: {penalidade_desequilibrio}")
    print(f"Aptidão final: {aptidao}")
    print("=================================\n")
    
    return aptidao

In [ ]:
teste_solu_aptidao = criar_solucao_inicial(N_ENFERMEIRAS, N_DIAS, N_TURNOS)
print(teste_solu_aptidao)

In [ ]:
calcular_aptidao(teste_solu_aptidao, requisicoes_turnos, N_ENFERMEIRAS, N_DIAS, N_TURNOS, MAX_TURNOS_SEMANAIS)

In [ ]:
calcular_aptidao_detalhada(teste_solu_aptidao, requisicoes_turnos, N_ENFERMEIRAS, N_DIAS, N_TURNOS, MAX_TURNOS_SEMANAIS)

### 5. Operadores Genéticos e geração de nova população
# 
- Seleção por Torneio: Seleciona os pais para a próxima geração com base em sua aptidão em torneios.
- Crossover de Ordem: Combina informações dos pais para gerar filhos, mantendo a ordem dos turnos.
- Taxa de Mutação Adaptativa: Ajusta a taxa de mutação com base na aptidão da solução.
- Mutação de Bloco: Aplica mutações a blocos de turnos na solução.
- Geração de Nova População: Combina os operadores genéticos para criar uma nova população

In [ ]:
def selecao_torneio(populacao, aptidoes, tamanho_torneio):
    """Seleciona um indivíduo da população usando seleção por torneio (vetorizado)."""

    aptidoes = np.array(aptidoes)  # Garante que aptidoes seja um array NumPy
    num_torneios = len(populacao) // 2   # Calcula o número de torneios (metade da população)+ 1
    indices = np.random.choice(len(populacao), size=(num_torneios, tamanho_torneio))  # Escolhe índices aleatórios para o torneio
    vencedores = np.argmax(aptidoes[indices], axis=1)  # Determina os vencedores do torneio
    return np.array(populacao)[indices[np.arange(num_torneios), vencedores]]  # Retorna os indivíduos vencedores

In [ ]:
aptidoes_ex = [calcular_aptidao(solucao, requisicoes_turnos, N_ENFERMEIRAS, N_DIAS, N_TURNOS, MAX_TURNOS_SEMANAIS) for solucao in população_ex]
print(aptidoes_ex)

In [ ]:
pais1_ex = selecao_torneio(população_ex, aptidoes_ex, TOURNAMENT_SIZE)
pais2_ex = selecao_torneio(população_ex, aptidoes_ex, TOURNAMENT_SIZE)

In [ ]:
pais1_ex

In [ ]:
def crossover_ordem(pai1, pai2, n_dias, n_turnos, n_enfermeiras):
    """Crossover aprimorado que utiliza informações de ambos os pais."""

    pai1 = np.array(pai1)  # Converte o pai1 para um array NumPy para manipulação eficiente
    pai2 = np.array(pai2)  # Converte o pai2 para um array NumPy para manipulação eficiente
    inicio, fim = sorted(np.random.randint(0, n_dias, size=2))  # Gera dois pontos de corte aleatórios e os ordena
    filho1 = np.zeros((n_dias, n_turnos), dtype=int) - 1  # Inicializa o filho1 com -1 (sem turnos atribuídos)
    filho2 = np.zeros((n_dias, n_turnos), dtype=int) - 1  # Inicializa o filho2 com -1 (sem turnos atribuídos)
    filho1[inicio:fim] = pai1[inicio:fim]  # Copia a seção do pai1 para o filho1
    filho2[inicio:fim] = pai2[inicio:fim]  # Copia a seção do pai2 para o filho2

    for dia in range(n_dias):  # Itera sobre os dias
        if dia < inicio or dia >= fim:  # Verifica se o dia está fora da seção de crossover
            for turno in range(n_turnos):  # Itera sobre os turnos do dia
                if filho1[dia, turno] == -1:  # Verifica se o turno não está atribuído no filho1
                    enfermeira_pai2 = pai2[dia, turno]  # Pega a enfermeira do pai2 para este turno
                    if enfermeira_pai2 not in filho1[dia]:  # Verifica se a enfermeira não está já atribuída no filho1 neste dia
                        filho1[dia, turno] = enfermeira_pai2  # Atribui a enfermeira do pai2 ao turno do filho1
                    else:
                        # Procura outra enfermeira válida (exemplo simples: próxima enfermeira)
                        enfermeira_alternativa = (enfermeira_pai2 + 1) % n_enfermeiras  # Calcula a próxima enfermeira disponível
                        while enfermeira_alternativa in filho1[dia]:  # Loop para encontrar uma enfermeira não usada
                            enfermeira_alternativa = (enfermeira_alternativa + 1) % n_enfermeiras  # Avança para a próxima enfermeira
                        filho1[dia, turno] = enfermeira_alternativa  # Atribui a enfermeira alternativa ao turno do filho1

                if filho2[dia, turno] == -1:  # Repete o processo para o filho2, trocando os papéis dos pais
                    enfermeira_pai1 = pai1[dia, turno]
                    if enfermeira_pai1 not in filho2[dia]:
                        filho2[dia, turno] = enfermeira_pai1
                    else:
                        enfermeira_alternativa = (enfermeira_pai1 + 1) % n_enfermeiras
                        while enfermeira_alternativa in filho2[dia]:
                            enfermeira_alternativa = (enfermeira_alternativa + 1) % n_enfermeiras
                        filho2[dia, turno] = enfermeira_alternativa
    return filho1.tolist(), filho2.tolist()  # Retorna os filhos como listas de listas

In [ ]:
filho1_ex, filho2_ex = crossover_ordem(pais1_ex[0], pais2_ex[0], N_DIAS, N_TURNOS, N_ENFERMEIRAS)

In [ ]:
print(pais1_ex[0])
print(pais2_ex[0])

In [ ]:
print(filho1_ex)
print(filho2_ex)

taxa_mutacao_base=0.05, intensidade_maxima=0.15

In [ ]:
def taxa_mutacao_adaptativa(aptidao, aptidao_min, aptidao_max, taxa_mutacao_base=0.05, intensidade_maxima=0.15):
    """
    Ajusta a taxa de mutação com base na aptidão da solução.
    Soluções com aptidão menor têm taxas de mutação maiores.
    """
    
    if aptidao_max == aptidao_min:  # Verifica se a aptidão máxima é igual à mínima
        return taxa_mutacao_base  # Retorna a taxa de mutação base para evitar divisão por zero

    normalizado = (aptidao - aptidao_min) / (aptidao_max - aptidao_min)  # Normaliza a aptidão para o intervalo [0, 1]
    intensidade = 1 - normalizado  # Calcula a intensidade da mutação (inverso da aptidão normalizada)
    return taxa_mutacao_base + intensidade * (intensidade_maxima - taxa_mutacao_base)  # Calcula a taxa de mutação ajustada

In [ ]:
def mutacao_bloco(solucao, taxa_mutacao, n_enfermeiras, n_dias, n_turnos):
    """Aplica mutação de bloco a uma solução usando NumPy."""

    tamanho_bloco = random.randint(0, n_turnos)  # Calcula um tamanho de bloco aleatório entre 0 e n_turnos
    solucao_mutada = np.array(solucao).copy()  # Cria uma cópia da solução para evitar modificar a original
    if random.random() < taxa_mutacao:  # Verifica se a mutação deve ocorrer com base na taxa de mutação
        dia = random.randint(0, n_dias - 1)  # Seleciona um dia aleatório para mutação
        inicio_bloco = random.randint(0, n_turnos - tamanho_bloco)  # Seleciona o início do bloco para mutação
        fim_bloco = inicio_bloco + tamanho_bloco  # Calcula o fim do bloco para mutação
        # Atribuindo valores válidos de enfermeiras
        solucao_mutada[dia, inicio_bloco:fim_bloco] = np.random.randint(0, n_enfermeiras, size=tamanho_bloco)  # Atribui novas enfermeiras ao bloco
    return solucao_mutada.tolist()  # Retorna a solução mutada como uma lista

In [ ]:
mutacao_bloco(filho1_ex, 1,  N_ENFERMEIRAS, N_DIAS, N_TURNOS)

In [ ]:
def gerar_nova_populacao(populacao, aptidoes, tamanho_populacao, tamanho_torneio, n_enfermeiras, n_dias,
                         n_turnos, aptidao_min, aptidao_max, requisicoes_turnos):
    """Gera uma nova população aplicando seleção, crossover e mutação (vetorizado)."""
    
    nova_populacao = [max(zip(populacao, aptidoes), key=lambda x: x[1])[0]]  # Mantém o melhor indivíduo da população atual, elitismo
    pais1 = selecao_torneio(populacao, aptidoes, tamanho_torneio)  # Seleciona pais usando seleção por torneio
    pais2 = selecao_torneio(populacao, aptidoes, tamanho_torneio)  # Seleciona pais usando seleção por torneio
    
    # Correção: Adiciona n_enfermeiras na chamada de crossover_ordem
    filhos1, filhos2 = zip(*[crossover_ordem(p1, p2, n_dias, n_turnos, n_enfermeiras) for p1, p2 in zip(pais1, pais2)])  # Aplica crossover aos pais
    filhos1 = list(filhos1)  # Converte os filhos1 de tupla para lista
    filhos2 = list(filhos2)  # Converte os filhos2 de tupla para lista
    
    taxas_mutacao1 = [taxa_mutacao_adaptativa(calcular_aptidao(filho, requisicoes_turnos, n_enfermeiras, n_dias, n_turnos, MAX_TURNOS_SEMANAIS), aptidao_min, aptidao_max) for filho in filhos1]  # Calcula taxas de mutação adaptativas para filhos1
    taxas_mutacao2 = [taxa_mutacao_adaptativa(calcular_aptidao(filho, requisicoes_turnos, n_enfermeiras, n_dias, n_turnos, MAX_TURNOS_SEMANAIS), aptidao_min, aptidao_max) for filho in filhos2]  # Calcula taxas de mutação adaptativas para filhos2
    
    mutantes1 = [mutacao_bloco(filho, taxa_mutacao, n_enfermeiras, n_dias, n_turnos) for filho, taxa_mutacao in zip(filhos1, taxas_mutacao1)]  # Aplica mutação de bloco aos filhos1
    mutantes2 = [mutacao_bloco(filho, taxa_mutacao, n_enfermeiras, n_dias, n_turnos) for filho, taxa_mutacao in zip(filhos2, taxas_mutacao2)]  # Aplica mutação de bloco aos filhos2

    nova_populacao.extend(mutantes1)  # Adiciona mutantes1 à nova população
    nova_populacao.extend(mutantes2)  # Adiciona mutantes2 à nova população
    
    return nova_populacao[:tamanho_populacao]  # Retorna a nova população, limitada ao tamanho desejado

### 6. Execução do Algoritmo Genético
# 
#### Mantém uma população evoluindo por várias gerações até parar de modificar o valor de apatidão por algumas gerações

### Migração Dinâmica:

Implementa uma estratégia de migração para evitar a convergência prematura do algoritmo.
#
Introduz novas soluções aleatórias na população após um certo número de gerações sem melhoria.

In [ ]:
def algoritmo_genetico(tamanho_populacao, n_convergencia, tamanho_torneio, n_enfermeiras, n_dias, n_turnos, max_turnos_semanais, requisicoes_turnos):
    """Executa o algoritmo genético com migração dinâmica."""

    tempo_inicial = time.time()  # Marca o tempo de início da execução
    populacao = criar_populacao_inicial(tamanho_populacao, n_enfermeiras, n_dias, n_turnos, requisicoes_turnos)  # Cria a população inicial
    melhor_aptidao_ate_agora = float('-inf')  # Inicializa a melhor aptidão encontrada até agora com um valor negativo infinito
    contador_sem_melhora = 0  # Inicializa o contador de gerações sem melhora
    geracao = 0  # Inicializa o contador de gerações
    historico_aptidao = []  # Inicializa a lista para armazenar o histórico de aptidões
    geracoes_sem_melhora_migracao = 0  # Inicializa o contador de gerações sem melhora para a migração
    migracoes_realizadas = 0  # Inicializa o contador de migrações realizadas

    while contador_sem_melhora < n_convergencia:  # Loop principal do algoritmo genético
        aptidoes = [calcular_aptidao(solucao, requisicoes_turnos, n_enfermeiras, n_dias, n_turnos, max_turnos_semanais) for solucao in populacao]  # Calcula a aptidão de cada solução na população
        melhor_solucao = max(zip(populacao, aptidoes), key=lambda x: x[1])[0]  # Encontra a melhor solução na população
        melhor_aptidao = calcular_aptidao(melhor_solucao, requisicoes_turnos, n_enfermeiras, n_dias, n_turnos, max_turnos_semanais)  # Calcula a aptidão da melhor solução
        historico_aptidao.append(melhor_aptidao)  # Adiciona a melhor aptidão ao histórico

        print(f"Geração {geracao + 1}: Aptidão = {melhor_aptidao}")  # Imprime a geração e a melhor aptidão

        if melhor_aptidao > melhor_aptidao_ate_agora:  # Verifica se a melhor aptidão melhorou
            melhor_aptidao_ate_agora = melhor_aptidao  # Atualiza a melhor aptidão encontrada até agora
            contador_sem_melhora = 0  # Reseta o contador de gerações sem melhora
            geracoes_sem_melhora_migracao = 0  # Reseta o contador de gerações sem melhora para a migração
            migracoes_realizadas = 0  # Reseta o contador de migrações
        else:
            contador_sem_melhora += 1  # Incrementa o contador de gerações sem melhora
            geracoes_sem_melhora_migracao += 1  # Incrementa o contador de gerações sem melhora para a migração

        if geracoes_sem_melhora_migracao >= 8:  # Verifica se é hora de realizar a migração
            num_migrantes = int(tamanho_populacao * (0.1 + migracoes_realizadas * 0.05))  # Calcula o número de migrantes
            num_migrantes = min(num_migrantes, tamanho_populacao)  # Garante que o número de migrantes não exceda o tamanho da população

            for _ in range(num_migrantes):  # Realiza a migração
                populacao[random.randint(0, tamanho_populacao - 1)] = criar_solucao_inicial(n_enfermeiras, n_dias, n_turnos)  # Substitui soluções aleatórias por novas soluções

            geracoes_sem_melhora_migracao = 0  # Reseta o contador de gerações sem melhora para a migração
            migracoes_realizadas += 1  # Incrementa o contador de migrações
            print(f"Migração realizada! Número de migrantes: {num_migrantes}")  # Imprime informações sobre a migração

        aptidao_min = min(aptidoes)  # Encontra a aptidão mínima na população
        aptidao_max = max(aptidoes)  # Encontra a aptidão máxima na população
        populacao = gerar_nova_populacao(populacao, aptidoes, tamanho_populacao, tamanho_torneio, n_enfermeiras, n_dias, n_turnos, aptidao_min, aptidao_max, requisicoes_turnos)  # Gera a nova população
        geracao += 1  # Incrementa o contador de gerações
    
    tempo_final = time.time()  # Marca o tempo de fim da execução
    tempo_execucao = tempo_final - tempo_inicial  # Calcula o tempo de execução
    return melhor_solucao, melhor_aptidao, historico_aptidao, tempo_execucao  # Retorna a melhor solução, sua aptidão, o histórico de aptidões e o tempo de execução

In [ ]:
# Execução do algoritmo
melhor_solucao, melhor_aptidao, historico_aptidao, tempo_execucao_genai = algoritmo_genetico(POPULATION_SIZE, N_CONVERGENCIA, TOURNAMENT_SIZE, N_ENFERMEIRAS, N_DIAS, N_TURNOS, MAX_TURNOS_SEMANAIS, requisicoes_turnos)

### 7. Melhor Solução Encontrada
# 
### Exibe a melhor solução após todas as gerações.

In [ ]:
print("\nMelhor solução encontrada:")
for dia in range(N_DIAS):
    print(f"Dia {dia}: {melhor_solucao[dia]}")
print(f"Aptidão da melhor solução: {melhor_aptidao}")
print(f"Tempo de execução do algoritmo genético: {tempo_execucao_genai:.4f} segundos")

In [ ]:
print(calcular_aptidao_detalhada(melhor_solucao, requisicoes_turnos, N_ENFERMEIRAS, N_DIAS, N_TURNOS, MAX_TURNOS_SEMANAIS))

In [ ]:
# Visualização do progresso
def plotar_evolucao(historico_aptidao):
    plt.figure(figsize=(10, 5))
    plt.plot(historico_aptidao, label='Melhor Aptidão', color='blue')
    plt.xlabel('Geração')
    plt.ylabel('Aptidão')
    plt.title('Evolução da Aptidão ao Longo das Gerações')
    plt.legend()
    plt.grid()
    plt.show()

plotar_evolucao(historico_aptidao)

### 8. Comparação do algoritmo desenvolvido com um algoritmo de força bruta
#
### Este trecho de código compara o desempenho de dois algoritmos para resolver um problema de alocação de enfermeiras: um algoritmo de força bruta e um algoritmo genético.

#### Número de Possibilidades no Método de Força Bruta

**Fórmula Geral:**

```
Possibilidades = (n_enfermeiras)^(n_dias * n_turnos)
```

**Exemplos:**

1.  **n_enfermeiras = 4, n_dias = 4, n_turnos = 4**

    * Possibilidades = 4^(4 \* 4) = 4^16 = 4.294.967.296

2.  **n_enfermeiras = 3, n_dias = 3, n_turnos = 3**

    * Possibilidades = 3^(3 \* 3) = 3^9 = 19.683

**Conclusão:**

O número de soluções cresce exponencialmente, tornando a força bruta inviável para problemas maiores.


In [ ]:
def forca_bruta(requisicoes_turnos, n_enfermeiras, n_dias, n_turnos, max_turnos_semanais, aptidao_alvo):
    """Resolve o problema de alocação de enfermeiras usando força bruta."""

    melhor_solucao = None  # Inicializa a melhor solução encontrada
    melhor_aptidao = float("-inf")  # Inicializa a melhor aptidão encontrada
    tempo_inicial = time.time()  # Marca o tempo de início da execução

    # Gera todas as combinações possíveis
    for solucao in itertools.product(range(n_enfermeiras), repeat=n_dias * n_turnos):
        solucao_matriz = [
            list(solucao[i * n_turnos : (i + 1) * n_turnos]) for i in range(n_dias)
        ]  # Converte a solução linear em uma matriz de dias x turnos
        aptidao = calcular_aptidao(
            solucao_matriz,
            requisicoes_turnos,
            n_enfermeiras,
            n_dias,
            n_turnos,
            max_turnos_semanais,
        )  # Calcula a aptidão da solução

        if aptidao > melhor_aptidao:  # Verifica se a aptidão é melhor que a melhor aptidão encontrada
            melhor_aptidao = aptidao  # Atualiza a melhor aptidão
            melhor_solucao = solucao_matriz  # Atualiza a melhor solução
            
        if aptidao >= aptidao_alvo:  # Verifica se a aptidão atingiu o alvo
            tempo_final = time.time()  # Marca o tempo de fim da execução
            tempo_execucao = tempo_final - tempo_inicial  # Calcula o tempo de execução
            return melhor_solucao, aptidao, tempo_execucao  # Retorna a melhor solução, aptidão e tempo de execução

    tempo_final = time.time()  # Marca o tempo de fim da execução
    tempo_execucao = tempo_final - tempo_inicial  # Calcula o tempo de execução

    return melhor_solucao, melhor_aptidao, tempo_execucao  # Retorna a melhor solução, aptidão e tempo de execução

In [ ]:
# Variáveis para armazenar os tempos de execução
tempos_forca_bruta = []  # Lista para armazenar os tempos de execução do algoritmo de força bruta
tempos_genetico = []  # Lista para armazenar os tempos de execução do algoritmo genético
diferenca_tempos = []  # Lista para armazenar as diferenças de tempo entre os algoritmos

In [ ]:
random.seed(42)
np.random.seed(42)

# Variação dos parâmetros (N_ENFERMEIRAS, N_DIAS e N_TURNOS)
valores_parametros = range(1, 8)  # Define um intervalo de valores para os parâmetros (de 1 a 4)

for valor in valores_parametros:  # Loop para iterar sobre os valores dos parâmetros
    # Ajuste dos parâmetros para a iteração atual
    print(valor)  # Imprime o valor atual dos parâmetros
    n_enfermeiras = valor  # Define o número de enfermeiras
    n_dias = valor  # Define o número de dias
    n_turnos = valor  # Define o número de turnos

    if valor >= 4: # redução da quantidade de possibilidades para não exigir muito tempo do computador
         n_dias = 3
         n_turnos =3

    n_convergencia = min(20, int(n_dias * 1.5))  # Define o número de gerações para convergência no algoritmo genético
    population_size = min(1000, 100 * n_dias + 10 * n_enfermeiras + 50 * n_turnos)  # Define o tamanho da população no algoritmo genético
    tournament_size = max(2, int(0.02 * population_size))  # Define o tamanho do torneio no algoritmo genético

    # Criação de dados de exemplo para requisicoes_turnos, garantindo variação com os parâmetros
    requisicoes_turnos = gerar_requisicoes_turnos(n_enfermeiras, n_dias, n_turnos)  # Gera as requisições de turnos

    # Algoritmo Genético
    tempo_inicial_genetico = time.time()  # Marca o tempo de início do algoritmo genético
    melhor_solucao_genetico, melhor_aptidao_genetico, historico_aptidao_genetico, tempo_genetico = algoritmo_genetico(
    population_size, n_convergencia, tournament_size, n_enfermeiras, n_dias, n_turnos, MAX_TURNOS_SEMANAIS, requisicoes_turnos)  # Executa o algoritmo genético
    tempo_final_genetico = time.time()  # Marca o tempo de fim do algoritmo genético
    tempo_genetico_exec = tempo_final_genetico - tempo_inicial_genetico  # Calcula o tempo de execução do algoritmo genético
    tempos_genetico.append(tempo_genetico_exec)  # Armazena o tempo de execução do algoritmo genético

    # Força Bruta
    tempo_inicial_forca_bruta = time.time()  # Marca o tempo de início do algoritmo de força bruta
    melhor_solucao_forca_bruta, melhor_aptidao_forca_bruta, tempo_forca_bruta = forca_bruta(
        requisicoes_turnos, n_enfermeiras, n_dias, n_turnos, MAX_TURNOS_SEMANAIS, melhor_aptidao_genetico)  # Executa o algoritmo de força bruta
    tempo_final_forca_bruta = time.time()  # Marca o tempo de fim do algoritmo de força bruta
    tempo_forca_bruta_exec = tempo_final_forca_bruta - tempo_inicial_forca_bruta  # Calcula o tempo de execução do algoritmo de força bruta
    tempos_forca_bruta.append(tempo_forca_bruta_exec)  # Armazena o tempo de execução do algoritmo de força bruta

    # Cálculo da diferença de tempos
    diferenca = tempo_forca_bruta_exec - tempo_genetico_exec  # Calcula a diferença de tempo entre os algoritmos
    diferenca_tempos.append(diferenca)  # Armazena a diferença de tempo

In [ ]:
# Impressão dos valores da diferença de tempo com melhor formatação
print("Diferenças de Tempo (Força Bruta - Genético):")  # Imprime um cabeçalho informativo
for i, diferenca in enumerate(diferenca_tempos):  # Loop para iterar sobre as diferenças de tempo
    tempo_forca_bruta = tempos_forca_bruta[i]  # Obtém o tempo de execução do algoritmo de força bruta
    tempo_genetico = tempos_genetico[i]  # Obtém o tempo de execução do algoritmo genético
    print(f"Parâmetros: {valores_parametros[i]}:")  # Imprime o valor dos parâmetros
    print(f"  Força Bruta: {tempo_forca_bruta:.4f} segundos")  # Imprime o tempo de execução do algoritmo de força bruta
    print(f"  Genético: {tempo_genetico:.4f} segundos")  # Imprime o tempo de execução do algoritmo genético
    print(f"  Diferença: {diferenca:.4f} segundos")  # Imprime a diferença de tempo entre os algoritmos
    print("-" * 30)  # Imprime um separador para melhor visualização

# Análises de Resultados:

- Comparação realizada com mesma semente aleatória e valor de aptidão.

## Resultados Principais:

- Problemas Pequenos (Parâmetros 1 e 2):
    - Força bruta executa quase instantaneamente.
    - Algoritmo genético tem sobrecarga inicial, sendo mais lento.
- Problemas Médios a Grandes (Parâmetros 3+):
    - Tempo da força bruta cresce exponencialmente.
    - Algoritmo genético mantém tempo de execução estável.
    - Diferença de tempo aumenta drasticamente, favorecendo o genético.
- Escalabilidade:
    - Algoritmo genético é significativamente mais escalável.

In [ ]:
# Visualização dos resultados em um gráfico
plt.figure(figsize=(10, 6))
plt.plot(valores_parametros, tempos_forca_bruta, label='Força Bruta', marker='o')
plt.plot(valores_parametros, tempos_genetico, label='Genético', marker='x')
plt.xlabel('Valor dos Parâmetros (Enfermeiras, Dias, Turnos)')
plt.ylabel('Tempo de Execução (segundos)')
plt.title('Comparação de Tempos de Execução')
plt.legend()
plt.grid(True)
plt.show()

# Conclusões:

- O algoritmo genético implementado demonstra ser eficaz na resolução do problema de alocação de enfermeiras.
- A combinação de estratégias de inicialização e operadores genéticos contribui para a qualidade da solução.
- O algoritmo genético se mostrou muito mais eficiente que o algoritmo de força bruta, dado o tempo de execução e a qualidade de resultados.
- O algoritmo genético fornece uma solução de alta qualidade em um tempo razoável.
- O código fornece uma base sólida para futuras melhorias e adaptações a diferentes cenários de alocação de enfermeiras.

# Possíveis Melhorias:

- Implementar restrições mais complexas.
- Ter liberdade para ter mais de uma enfermeira por turno e poder relacionar a quantidade de enfermeiras com a quantidade de pacientes.
- Adaptar o algoritmo para lidar com mudanças dinâmicas na demanda por turnos.
- Implementar a possibilidade de mutações mais agressivas para testar os resultados
- Explorar diferentes estratégias de crossover e mutação para melhorar o desempenho.
- Implementar uma interface gráfica para facilitar a visualização e edição dos cronogramas.